# Measure Limbs

The aim of this notebook is to automatically take the same measurements as in the 1D dataset on the limbs created using the SSM. What this is going to lead to is the creation of a ML pipeline which will take in measurements and output the components of the SSM that get closest to that particular set of measurements (Also lets include an output for scale). Inshallah.

I think the approach I'm going to take for the circumference measurements is fairly simple - define a plane based on a vertical normal and a selected vertex. For every vert figure out which side of the plane its on. Find edges which cut the plane (And vertices on the plane). Find the intersection points between the edge and the plane. Join points based on face adjaceny - et voila. A curve on the plane. Then its just a matter of adding up all the line segments in a loop and bobs your uncle you have the circumfrential measurement. Maybe there is a nicer way to do it with a half edge mesh but I really haven't played about with that too much. 

As for the lengths: Just pick to verts and off you go. Nothing complicated about that at all.

In [19]:
import igl
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict

verts, face2vert = igl.read_triangle_mesh("limb_00000.stl")

# Remove duplicate vertices and update face2vert accordingly
# STL files often have duplicated vertices, so we need to merge them

# Use numpy structured array for unique operation
verts_rounded = np.round(verts, decimals=8)
_, unique_indices, inverse_indices = np.unique(verts_rounded, axis=0, return_index=True, return_inverse=True)

verts_unique = verts[unique_indices]
face2vert_unique = inverse_indices[face2vert]

verts = verts_unique
face2vert = face2vert_unique

edge2vert, face2edge, edge2face = igl.edge_topology(verts, face2vert)

## Slicing out a curve with a plane
So we're going to try get out a 2d curve out of this mesh. So one of the things I currently cant remember how to do is find where this edge intersects a plane but lets work it out because thats fun. 

We start with a point on the edge which can be described as

$$p_t = tv_1 + (1 - t)v_2$$

We then want to find when the vector from this to the plane point is orthogonal so: 

$$(p_t - p_p) \cdot n_p = 0$$

Which when you look at it you realise is really fucking easy because in the case of a vertical normal vector you end up with making the z value zero obviously. But if the plane wasnt horizontal then you'd just have to solve that equation and everyone will be happy. Great. So in the case of horizontal planes I should be solving: 

$$v_{2z} + (v_{1z} - v_{2z})t = 0$$

$$\frac{v_{2z}}{v_{2z} - v_{1z}} = t$$

In [20]:
plane_normal = np.array([0, 0, 1])
plane_point = np.array([0, 0, 0])

plane_side = np.dot(verts - plane_point, plane_normal)
# If a vert sits exactly on the plane then make it
plane_side[plane_side == 0] = 1e-8
plane_side =  plane_side < 0


edge_verts_side = plane_side[edge2vert]
edges_cross_plane = np.where(edge_verts_side[:, 0] != edge_verts_side[:, 1])[0]

edge_cut_verts = verts[edge2vert[edges_cross_plane]]

v1 = edge_cut_verts[:, 0]
v2 = edge_cut_verts[:, 1]
d1 = np.dot(v1 - plane_point, plane_normal)
d2 = np.dot(v2 - plane_point, plane_normal)
t = d1 / (d1 - d2)
intersection_points = v1 + (v2 - v1) * t[:, np.newaxis]

intersection_points.shape

with open("test_cuts.obj", "w") as f:
    for v1,v2,v3 in intersection_points:
        f.write(f"v {v1} {v2} {v3}\n")

In [21]:
edge_set = set(edges_cross_plane)

# For each edge crossing the plane, get the two faces it belongs to (ignore -1, which means no face)
faces_per_edge = edge2face[edges_cross_plane]  # shape (n_cross, 2)

# Flatten and filter out -1 to get all face indices involved in the cut
faces_in_cut = np.unique(faces_per_edge[faces_per_edge != -1])

# For each face in the cut, get its three edges
cut_face_edges = face2edge[faces_in_cut]  # shape (n_faces, 3)

# Only keep edges that are in edge_set (i.e., cross the plane)
cut_face_edges_mask = np.isin(cut_face_edges, list(edge_set))
cut_face_edges = np.where(cut_face_edges_mask, cut_face_edges, -1)

# For each face, the two edges with edge_set membership are the ones that cross the plane
# Find the indices of those edges for each face
connections = []
for face_edges in cut_face_edges:
    cut_edges = face_edges[face_edges != -1]
    if len(cut_edges) == 2:
        # Map edge index back to intersection_points index
        idx1 = np.where(edges_cross_plane == cut_edges[0])[0][0]
        idx2 = np.where(edges_cross_plane == cut_edges[1])[0][0]
        connections.append((idx1, idx2))
connections = np.array(connections)

In [22]:
def find_polyline(connections_dict, start):
    seen = {start}

    to_do = [*connections_dict[start]]
    path = [start]

    while to_do:
        curr = to_do.pop()
        path.append(curr)
        left, right = connections_dict[curr]

        if left not in seen:
            seen.add(left)
            to_do.append(left)
            continue

        if right not in seen:
            seen.add(right)
            to_do.append(right)
            continue

    return path

connections_dict = defaultdict(list)
for i, j in connections:
    i, j = int(i), int(j)
    connections_dict[i].append(j)
    connections_dict[j].append(i)

seen = set()
unseen = set(connections_dict.keys())

paths = []

while unseen:
    start = unseen.pop()
    paths.append(find_polyline(connections_dict, start))
    seen.union(set(paths[-1]))
    unseen -= set(paths[-1])

len(paths) 

4

In [23]:
with open("test_cuts.obj", "w") as f:
    for v1, v2, v3 in intersection_points:
        f.write(f"v {v1} {v2} {v3}\n")

    # Add edges between connected faces (as lines)
    # Add polylines from paths as lines in the obj file
    for p in paths:
        for i, vert in enumerate(p):
            f.write(f"l {p[i - 1] + 1} {vert + 1}\n")

In [24]:
def calculate_polyline_length(path, verts):
    path = np.array(path)

    return np.sum(np.linalg.norm(verts[path] - verts[np.roll(path, 1)], axis=1))

lengths = []
for line in paths:
    length = calculate_polyline_length(line, intersection_points)
    print(length)
    lengths.append(length)

print(np.pi*2)
print()
print([len(p) for p in paths])
print(sum([len(p) for p in paths]))


1.0016207488099733
0.22322107705881714
0.028186110751917428
0.15945071238592476
6.283185307179586

[231, 94, 13, 61]
399


# Trying a different approach

In [25]:
#| export_section
import torch
#| end_section

verts = torch.tensor(verts_unique)
face2vert = torch.tensor(face2vert_unique)

edge2vert, face2edge, edge2face = torch.tensor(edge2vert), torch.tensor(face2edge), torch.tensor(edge2face)


In [26]:
plane_normal = torch.tensor([[0, 0, 1]], dtype=verts.dtype)
plane_point = torch.tensor([[0, 0, 0]], dtype=verts.dtype)
plane_direction = torch.tensor([[1, 0, 0]], dtype=verts.dtype)


plane_side = torch.einsum("ij, ij -> i", verts - plane_point, plane_normal)

# If a vert sits exactly on the plane then make it
plane_side[plane_side == 0] = 1e-8
plane_side =  plane_side < 0



In [27]:
a, b = (torch.rand((2, 3), dtype=torch.float32) - 0.5) * 2
n = torch.tensor((0, 0, 1), dtype=torch.float32)
p = torch.tensor((0, 0, 0), dtype=torch.float32)


#| export
def plane_edge_intersection(a, b, n, p):
    if len(a.shape) == 2:
        t = torch.einsum("ij, ij -> i", p - a, n) / torch.einsum("ij, ij -> i", b - a, n)
        t = t.unsqueeze(-1)
    else:
        t = torch.dot(p - a, n) / torch.dot(b - a, n)
    return torch.nan_to_num(a + t * (b - a), nan=float('inf')), t


plane_edge_intersection(a, b, n, p)

(tensor([-0.2681, -0.2771,  0.0000]), tensor(0.4139))

In [28]:
edge_verts = verts[edge2vert]
intersections, t = plane_edge_intersection(edge_verts[:,0], edge_verts[:, 1], plane_normal, plane_point)
t_mask = torch.logical_and(0 <= t, t < 1)

torch.all(torch.isclose(intersections[:, -1], torch.zeros_like(intersections[:, -1])))

tensor(True)

In [29]:
face_intersections = intersections[face2edge]
face_distance_lines = torch.concat([
    (face_intersections[:,0] - face_intersections[:, 1])[:, None],
    (face_intersections[:,2] - face_intersections[:, 1])[:, None],
    (face_intersections[:,0] - face_intersections[:, 2])[:, None]
], dim=1)

face_distances = torch.linalg.norm(face_distance_lines, dim=-1, keepdim=True)

t_mask_faces_raw = t_mask[face2edge]
t_mask_faces = torch.concat([
    (t_mask_faces_raw[:, 0] + t_mask_faces_raw[:, 1] == 2)[:, None],
    (t_mask_faces_raw[:, 2] + t_mask_faces_raw[:, 1] == 2)[:, None],
    (t_mask_faces_raw[:, 0] + t_mask_faces_raw[:, 2] == 2)[:, None],
], dim=1)

torch.nansum(face_distances*t_mask_faces)

tensor(0., dtype=torch.float64)

In [30]:
# with open("torch_test_cuts.obj", "w") as f:
#     for face_ints, t_s in zip(face_intersections, t_mask_faces_raw):
#         for (v1, v2, v3), t in zip(face_ints, t_s):
#             if t:
#                 f.write(f"v {v1} {v2} {v3}\n")        

# torch.sum(t_mask_faces)

In [ ]:
#| export
def measure_planar_circumference(verts, edge2vert, face2edge, plane_point, plane_normal):
    if isinstance(plane_point, int):
        plane_point = verts[plane_point]

    if isinstance(plane_normal, (tuple, list)):
        plane_normal = verts[plane_normal[0]] - verts[plane_normal[1]]
        plane_normal /= torch.linalg.norm(plane_normal)

    edge_verts = verts[edge2vert]
    intersections, t = plane_edge_intersection(
        edge_verts[:, 0], edge_verts[:, 1], plane_normal, plane_point
    )

    t_mask = (0 < t) * (t < 1)

    face_intersections = intersections[face2edge]
    face_distances = torch.linalg.norm(
        torch.concat(
            [
                (face_intersections[:, 0] - face_intersections[:, 1])[:, None],
                (face_intersections[:, 2] - face_intersections[:, 1])[:, None],
                (face_intersections[:, 0] - face_intersections[:, 2])[:, None],
            ],
            dim=1,
        ),
        dim=-1,
        keepdim=True,
    )

    t_mask_faces = t_mask[face2edge]
    t_mask_faces = torch.concat(
        [
            (t_mask_faces[:, 0] * t_mask_faces[:, 1])[:, None],
            (t_mask_faces[:, 2] * t_mask_faces[:, 1])[:, None],
            (t_mask_faces[:, 0] * t_mask_faces[:, 2])[:, None],
        ],
        dim=1,
    )

    return torch.nansum(face_distances * t_mask_faces)

measure_planar_circumference(verts, edge2vert, face2edge, plane_point, plane_normal) - torch.pi*2

tensor(-4.8757, dtype=torch.float64)

### Other linear measurements

So basically the circumferential measurements should be the hardest but probably worth doing the widths in here as well. 

In [ ]:
#| export
def measure_width(verts, edge2vert, plane_point, plane_normal, plane_direction):
    if isinstance(plane_point, int):
        plane_point = verts[plane_point]

    if isinstance(plane_normal, (tuple, list)):
        plane_normal = verts[plane_normal[0]] - verts[plane_normal[1]]
        plane_normal /= torch.linalg.norm(plane_normal)

    if isinstance(plane_direction, (tuple, list)):
        plane_direction = verts[plane_direction[0]] - verts[plane_direction[1]]
        plane_direction /= torch.linalg.norm(plane_direction)

    edge_verts = verts[edge2vert]
    intersections, t = plane_edge_intersection(
        edge_verts[:, 0], edge_verts[:, 1], plane_normal, plane_point
    )

    t_mask = (0 < t) * (t < 1)

    valid_intersections = intersections[t_mask.squeeze()]

    values = torch.einsum("ij, ij -> i", valid_intersections, plane_direction)

    return torch.max(values) - torch.min(values)

measure_width(verts, edge2vert, plane_point, plane_normal, plane_direction)

tensor(0.3169, dtype=torch.float64)

In [ ]:
#| export
def measure_length(verts, v1, v2, direction):
    if isinstance(direction, (tuple, list)):
        direction = verts[direction[0]] - verts[direction[1]]
        direction /= torch.linalg.norm(direction)

    return torch.dot(verts[v1] - verts[v2], direction)

## Removing Bones from file
So we dont really want to include the size of the bones in our measurements so we're going to have to figure out how to exclude them from out model before it all goes terribly horribly wrong. So lets try isolate the bone verts so we can mask them out.

In [34]:
verts_bones, face2vert_bones = igl.read_triangle_mesh("limb_00000.stl")
verts_boneless, face2vert_boneless = igl.read_triangle_mesh("limb_00000_boneless.stl")

verts_bones, verts_boneless = np.round(verts_bones, decimals=8), np.round(verts_boneless, decimals=8)

_, boneless_idxs, inverse = np.unique(verts_boneless, return_index=True, return_inverse=True, axis=0)

_, bones_idxs = np.unique(verts_bones, return_index=True, axis=0)

verts_bones[bones_idxs]

boneless_set = {tuple(x):i for i, x in enumerate(verts_boneless[boneless_idxs])}
mapping = []
face_mapping = {}
for idx in bones_idxs:
    if tuple(verts_bones[idx]) in boneless_set:
        face_mapping[tuple(verts_bones[idx])] = len(mapping)
        mapping.append(idx)

mapping = np.array(mapping)
face2vert_boneless_mapped = np.vectorize(lambda x: face_mapping[tuple(verts_boneless[x])])(face2vert_boneless)

mapping = torch.tensor(mapping)
boneless_face2vert = torch.tensor(face2vert_boneless_mapped)

torch.save(mapping, "data_components/vert_mapping.pt")
torch.save(boneless_face2vert, "data_components/face2vert.pt")

In [35]:
#| export
def remove_bones(verts, path="/"):
    mapping = torch.load(path + "vert_mapping.pt")
    face2vert = torch.load(path + "face2vert.pt")

    return verts[mapping], face2vert

In [36]:
import nb_exporter
nb_exporter.export_notebook("measure_limbs.ipynb")